# IMPORTS

In [93]:
import numpy as np
import cv2
from skimage import io
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import VotingClassifier
from SkinDetection import *
from HandDetection import *
from Utilities import *
import pickle
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model Training

# Static images model

In [84]:
directories = [ 'images/AL/frame',          # 0
                'images/Alef/frame',        # 1
                'images/BB/frame',          # 2
                'images/dhad/frame',        # 3
                'images/four/frame',        # 4
                'images/ghain/frame',       # 5
                'images/jeem/frame',        # 6
                'images/LA/frame',          # 7
                'images/lam/frame',         # 8
                'images/O/frame',           # 9
                'images/okay/frame',        # 10
                'images/openpalm/frame',    # 11
                'images/peace/frame',       # 12
                'images/three/frame',       # 13
                'images/ya/frame',          # 14
                'images/starting/frame'     # 15
            ]
labels = list(range(0, len(directories)))
print(labels)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


1. Read Dataset
2. Apply Preproccessing
3. Extract Features

In [87]:
# df = pd.DataFrame(columns=['H[0]', 'H[1]','H[2]','H[3]','H[4]','H[5]','H[6]' ,'Label'])

dataset = []
for index in range(0,len(directories)):
    print(directories[index])
    for x in range(100,400):

        filename = directories[index] + str(x) + '.jpg'
        try:
            image = io.imread(filename)
            cv2_image = cv2.imread(filename)
        except:
            continue
        one = SkinDetection_InRange(cv2_image)
        two = SkinDetection_Multiscale(image)
        

        final = cv2.bitwise_and(one,two)

        _, hand = HandDetection(final)

        # cv2.imwrite("processed/frame" + str(x) + '.jpg', hand)

        huMoments = seven_hu_moments(hand)
        huMoments = abs(huMoments)
        huMoments = np.append(huMoments ,np.array(labels[index]))
        dataset.append(huMoments)
# #         df = df.append(pd.DataFrame(huMoments.reshape(1,-1), columns=list(df)), ignore_index=True)

# # print(df)
# print(dataset)

images/AL/frame


e:\kolya\Sana 4\Image Processing\Cleaner_Version\SkinDetection.py:114: RuntimeWarning: divide by zero encountered in true_divide
  check = np.bitwise_and(red_dash/green_dash > 1.185, np.bitwise_and(cb > 77, np.bitwise_and(cb < 127,np.bitwise_and(cr > 133, np.bitwise_and(cb < 173 , np.bitwise_and(saturation >= 0.2 , np.bitwise_and(saturation<= 0.6 , np.bitwise_and(hue >=0 , np.bitwise_and( hue <= 25 , np.bitwise_or(hue >= 335 , hue <= 360))))))))))
e:\kolya\Sana 4\Image Processing\Cleaner_Version\SkinDetection.py:69: RuntimeWarning: invalid value encountered in true_divide
  red_dash = red/(red+green+blue)
e:\kolya\Sana 4\Image Processing\Cleaner_Version\SkinDetection.py:70: RuntimeWarning: invalid value encountered in true_divide
  green_dash = green/(red+green+blue)
e:\kolya\Sana 4\Image Processing\Cleaner_Version\SkinDetection.py:71: RuntimeWarning: invalid value encountered in true_divide
  blue_dash = blue/(red+green+blue)
e:\kolya\Sana 4\Image Processing\Cleaner_Version\SkinDetect

images/Alef/frame
images/BB/frame
images/dhad/frame
images/four/frame
images/ghain/frame
images/jeem/frame
images/LA/frame
images/lam/frame
images/O/frame
images/okay/frame
images/openpalm/frame
images/peace/frame
images/three/frame
images/ya/frame
images/starting/frame


Classifier - VotingClassifier (SVM+KNN)

In [92]:
data_array = np.array(dataset)
print(data_array.shape)
y = data_array[:, -1]
x = data_array[:, :-1]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 1/3, random_state=42)

clf = svm.SVC(probability=True ,kernel = 'rbf', gamma=1/3, C=1500)
clf.fit(X_train, y_train)
print('SVM' , clf.score(X_test, y_test))

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
print('KNN' , knn.score(X_test, y_test))

static_combine = VotingClassifier(estimators=[('SVM',clf) ,('KNN',knn)],voting='soft',weights = [2,1])
static_combine.fit(X_train, y_train)
print('Combined Voting' ,static_combine.score(X_test, y_test))

(3607, 8)
SVM 0.8761429758935994
KNN 0.7913549459684123
Combined Voting 0.8711554447215295


## Store the models for later use

In [ ]:
filename = 'finalized_model.sav'
pickle.dump(static_combine, open(filename, 'wb'))

## Live Testing

In [ ]:
# loaded_model = pickle.load(open('finalized_model.sav', 'rb'))
cap = cv2.VideoCapture(0)
count = 0

while True:
    time.sleep(1)
    _,frame = cap.read()
    
    cv2.imwrite('Khaled/frame'+str(count) +'.jpg', frame)
    image = io.imread('Khaled/frame'+str(count) +'.jpg')
    # count += 1

    one = SkinDetection_InRange(frame)
    two = SkinDetection_Multiscale(image)
    final = cv2.bitwise_and(one,two)
    binary, hand = HandDetection(final)

    huMoments = seven_hu_moments(hand)
    huMoments = abs(huMoments)
    testing = np.array(huMoments)

    label = int(static_combine.predict(testing.T))
    print(directories[label])
    # if (label == 
    cv2.imshow("Main",frame)
    # cv2.imshow("IO",image)
    # cv2.imshow("One",one)
    # cv2.imshow("Two",two)
    cv2.imshow("Final",binary)
    cv2.imshow("Hand",hand)
    
    if cv2.waitKey(1)& 0xFF == ord('q'):
        break

#Releasing the camera
cap.release()
cv2.destroyAllWindows()


## Sequence Based

In [94]:
dir=[[directories[0],directories[2]],[directories[8],directories[6]],[directories[3],directories[14]]]

seq_dataset=[]

seq_based_labels=[]

for i in range(len(dir)):

    for j in range(100,400):
        one_sequence = np.zeros((7*len(dir[0])))
        start_slicing=0
        end_slicing=7

        for fr in dir[i]:

            filename = fr + str(j) + '.jpg'

            try:
                image = io.imread(filename)
                cv2_image = cv2.imread(filename)
            except:
                break

            one = SkinDetection_InRange(cv2_image)
            two = SkinDetection_Multiscale(image)

            final = cv2.bitwise_and(one,two)

            _, hand = HandDetection(final)

            huMoments = seven_hu_moments(hand)
            huMoments = abs(huMoments)

            one_sequence[start_slicing:end_slicing]=huMoments.T

        
        seq_dataset.append(one_sequence)

        seq_based_labels.append(i)
            
            # start_slicing+=end_slicing
            # end_slicing+=end_slicing
           

        
        
print(np.array(seq_based_labels).shape)

e:\kolya\Sana 4\Image Processing\Cleaner_Version\SkinDetection.py:114: RuntimeWarning: divide by zero encountered in true_divide
  check = np.bitwise_and(red_dash/green_dash > 1.185, np.bitwise_and(cb > 77, np.bitwise_and(cb < 127,np.bitwise_and(cr > 133, np.bitwise_and(cb < 173 , np.bitwise_and(saturation >= 0.2 , np.bitwise_and(saturation<= 0.6 , np.bitwise_and(hue >=0 , np.bitwise_and( hue <= 25 , np.bitwise_or(hue >= 335 , hue <= 360))))))))))
e:\kolya\Sana 4\Image Processing\Cleaner_Version\SkinDetection.py:114: RuntimeWarning: invalid value encountered in true_divide
  check = np.bitwise_and(red_dash/green_dash > 1.185, np.bitwise_and(cb > 77, np.bitwise_and(cb < 127,np.bitwise_and(cr > 133, np.bitwise_and(cb < 173 , np.bitwise_and(saturation >= 0.2 , np.bitwise_and(saturation<= 0.6 , np.bitwise_and(hue >=0 , np.bitwise_and( hue <= 25 , np.bitwise_or(hue >= 335 , hue <= 360))))))))))
e:\kolya\Sana 4\Image Processing\Cleaner_Version\SkinDetection.py:69: RuntimeWarning: invalid val

(900,)


In [95]:
print(np.array(seq_dataset).shape)

(900, 14)


In [96]:
X_seq_train, X_seq_test, y_seq_train, y_seq_test = train_test_split(seq_dataset, seq_based_labels, 
                                                    test_size=0.2, 
                                                    random_state=111)

clf_seq = make_pipeline(StandardScaler(), SVC(gamma='scale', C=1000))

clf_seq.fit(X_seq_train,y_seq_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC(C=1000))])